In [2]:
import pandas as pd, numpy as np
import os
from tqdm import tqdm
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
import csv


def decode(a):
    if a =='A' or a =='2':
        return int(2)
    elif  a=='B' or a == '1':
        return int(1)
    elif a=='C' or a =='0':
        return int(0)
    else:
        return int(a)

patient_list=os.listdir('/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/')
import random
def split_list(lst, chunk_size):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]
print(len(patient_list))
for i in range(10):
    print(len(split_list(patient_list,len(patient_list)//10)[i]))

test0=split_list(patient_list,len(patient_list)//10)[0]
train0=list(set(patient_list)-set(test0))
train0.extend(split_list(patient_list,len(patient_list)//10)[10])
test1=split_list(patient_list,len(patient_list)//10)[1]
train1=list(set(patient_list)-set(test1))
train1.extend(split_list(patient_list,len(patient_list)//10)[10])
test2=split_list(patient_list,len(patient_list)//10)[2]
train2=list(set(patient_list)-set(test2))
train2.extend(split_list(patient_list,len(patient_list)//10)[10])
test3=split_list(patient_list,len(patient_list)//10)[3]
train3=list(set(patient_list)-set(test3))
train3.extend(split_list(patient_list,len(patient_list)//10)[10])
test4=split_list(patient_list,len(patient_list)//10)[4]
train4=list(set(patient_list)-set(test4))
train4.extend(split_list(patient_list,len(patient_list)//10)[10])
test5=split_list(patient_list,len(patient_list)//10)[5]
train5=list(set(patient_list)-set(test5))
train5.extend(split_list(patient_list,len(patient_list)//10)[10])
test6=split_list(patient_list,len(patient_list)//10)[6]
train6=list(set(patient_list)-set(test6))
train6.extend(split_list(patient_list,len(patient_list)//10)[10])
test7=split_list(patient_list,len(patient_list)//10)[7]
train7=list(set(patient_list)-set(test7))
train7.extend(split_list(patient_list,len(patient_list)//10)[10])
test8=split_list(patient_list,len(patient_list)//10)[8]
train8=list(set(patient_list)-set(test8))
train8.extend(split_list(patient_list,len(patient_list)//10)[10])
test9=split_list(patient_list,len(patient_list)//10)[9]
train9=list(set(patient_list)-set(test9))
train9.extend(split_list(patient_list,len(patient_list)//10)[10])


1489
148
148
148
148
148
148
148
148
148
148


In [4]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced') #predicted prob for fre grade
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced') #predicted prob for fro grade
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')#predicted prob for frefro grades


train_label=[]
ii=0
for f in train0:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['POutc'][i]==6:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)  #fresh grade
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)  # frozen grade 
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True) # fresh and frozen grade



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test0:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)

    y_test=np.where((p['POutc']).to_numpy()==6,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fre/fold0_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fro/fold0_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_frefro/fold0_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)


In [5]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train1:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['POutc'][i]==6:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test1:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)

    y_test=np.where((p['POutc']).to_numpy()==6,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fre/fold1_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fro/fold1_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_frefro/fold1_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)

In [6]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train2:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['POutc'][i]==6:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test2:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)

    y_test=np.where((p['POutc']).to_numpy()==6,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fre/fold2_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fro/fold2_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_frefro/fold2_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
       

In [7]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train3:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['POutc'][i]==6:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test3:

    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)

    y_test=np.where((p['POutc']).to_numpy()==6,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fre/fold3_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fro/fold3_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_frefro/fold3_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)      

In [8]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train4:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['POutc'][i]==6:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test4:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)


    y_test=np.where((p['POutc']).to_numpy()==6,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fre/fold4_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fro/fold4_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_frefro/fold4_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)

In [9]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train5:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['POutc'][i]==6:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test5:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)

    y_test=np.where((p['POutc']).to_numpy()==6,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fre/fold5_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fro/fold5_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_frefro/fold5_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)

In [10]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train6:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['POutc'][i]==6:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test6:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)

    y_test=np.where((p['POutc']).to_numpy()==6,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fre/fold6_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fro/fold6_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_frefro/fold6_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)      

In [11]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train7:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['POutc'][i]==6:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test7:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)


    y_test=np.where((p['POutc']).to_numpy()==6,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fre/fold7_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fro/fold7_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_frefro/fold7_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)

In [12]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train8:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['POutc'][i]==6:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test8:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)

    y_test=np.where((p['POutc']).to_numpy()==6,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fre/fold8_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fro/fold8_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_frefro/fold8_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)     

In [13]:
grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
Grf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')
ggrf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=27,class_weight='balanced')

train_label=[]
ii=0
for f in train9:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    for i in range(p.shape[0]):
        if p['POutc'][i]==6:
            train_label.append(1)
        else:
            train_label.append(0)
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)
    if ii==0:
        gtrain=c
        Gtrain=C
        ggtrain=cc
        ii=ii+1
    else:
        gtrain=pd.concat([gtrain,c], ignore_index=True)
        Gtrain=pd.concat([Gtrain,C], ignore_index=True)
        ggtrain=pd.concat([ggtrain,cc], ignore_index=True)



ytrain = np.array(train_label)

grf.fit(gtrain, ytrain)
Grf.fit(Gtrain, ytrain)
ggrf.fit(ggtrain, ytrain)

for f in test9:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    
    a=p[['g1','g2','g3']].replace('A',2)
    b=a.replace('B',1)
    c=b.replace('C',0)
    A=p[['G1','G2','G3']].replace('A',2)
    B=A.replace('B',1)
    C=B.replace('C',0)
    aa=p[['g1','g2','g3','G1','G2','G3']].replace('A',2)
    bb=aa.replace('B',1)
    cc=bb.replace('C',0)


    y_test=np.where((p['POutc']).to_numpy()==6,1,0)
    
    y_pred=grf.predict_proba(c)[:,1]
    y_pred_after_thre=grf.predict(c)
    
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fre/fold9_"+f, "w", newline='') as csvfile:

        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
            
    y_pred=Grf.predict_proba(C)[:,1]
    y_pred_after_thre=Grf.predict(C)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_fro/fold9_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)
            
    y_pred=ggrf.predict_proba(cc)[:,1]
    y_pred_after_thre=ggrf.predict(cc)
    with open("/mnt/louisayu/nfs_share2/embryo/code/em_aug/fropredict/result/pregprob_frefro/fold9_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j], y_pred_after_thre[j]]
            writer.writerow(content)   